In [1]:
from binance_trader import BinanceTrader
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")

trader = BinanceTrader(API_KEY, SECRET_KEY)
idx = pd.IndexSlice

In [2]:
params = {"symbol" : ["BTCUSDT", "ETHUSDT"], "interval" : "1m", "start_dt" : "2024-11-01", "end_dt" : "2024-12-22"}


df = trader.get_candlestick_data_range()


Open      High       Low     Close   Volume  \
Open Time           Symbol                                                     
2024-11-05 13:24:00 BTCUSDT  68710.01  68789.68  68710.01  68736.92  0.29141   
                    ETHUSDT   2438.40   2440.11   2438.40   2440.11  3.47320   
2024-11-05 13:25:00 BTCUSDT  68736.00  68755.48  21250.00  68750.26  0.32806   
                    ETHUSDT   2438.81   2439.92   2438.81   2439.53  3.22890   
2024-11-05 13:26:00 BTCUSDT  68746.87  69039.27  68666.48  68748.01  0.48238   
...                               ...       ...       ...       ...      ...   
2024-12-21 22:58:00 ETHUSDT   3323.34   3326.53   3323.34   3326.53  1.38570   
2024-12-21 22:59:00 BTCUSDT  97094.05  97108.00  97094.05  97108.00  0.03038   
                    ETHUSDT   3326.71   3329.00   3326.71   3329.00  4.30580   
2024-12-21 23:00:00 BTCUSDT  97108.04  97131.59  97108.04  97131.59  0.07146   
                    ETHUSDT   3328.33   3330.48   3328.33   3330.48  4.73910   

                             Quote Asset Volume  Number of Trades  \
Open Time           Symbol                                          
2024-11-05 13:24:00 BTCUSDT        20032.192563              56.0   
                    ETHUSDT         8473.003310              27.0   
2024-11-05 13:25:00 BTCUSDT        22434.253383              73.0   
                    ETHUSDT         7876.714896              36.0   
2024-11-05 13:26:00 BTCUSDT        33174.664570             131.0   
...                                         ...               ...   
2024-12-21 22:58:00 ETHUSDT         4606.946458              21.0   
2024-12-21 22:59:00 BTCUSDT         2949.964025              16.0   
                    ETHUSDT        14329.616799              55.0   
2024-12-21 23:00:00 BTCUSDT         6940.328925              36.0   
                    ETHUSDT        15780.074164              58.0   

                             Taker Buy Base Asset Volume  \
Open Time           Symbol                                 
2024-11-05 13:24:00 BTCUSDT                      0.29141   
                    ETHUSDT                      3.17170   
2024-11-05 13:25:00 BTCUSDT                      0.10787   
                    ETHUSDT                      3.04850   
2024-11-05 13:26:00 BTCUSDT                      0.30178   
...                                                  ...   
2024-12-21 22:58:00 ETHUSDT                      1.38570   
2024-12-21 22:59:00 BTCUSDT                      0.03038   
                    ETHUSDT                      4.30580   
2024-12-21 23:00:00 BTCUSDT                      0.07046   
                    ETHUSDT                      4.73220   

                             Taker Buy Quote Asset Volume  
Open Time           Symbol                                 
2024-11-05 13:24:00 BTCUSDT                  20032.192563  
                    ETHUSDT                   7737.543536  
2024-11-05 13:25:00 BTCUSDT                   7415.894900  
                    ETHUSDT                   7436.569564  
2024-11-05 13:26:00 BTCUSDT                  20755.789572  
...                                                   ...  
2024-12-21 22:58:00 ETHUSDT                   4606.946458  
2024-12-21 22:59:00 BTCUSDT                   2949.964025  
                    ETHUSDT                  14329.616799  
2024-12-21 23:00:00 BTCUSDT                   6843.220885  
                    ETHUSDT                  15757.108687  

[133634 rows x 9 columns]

In [4]:
for symbol in ["BTCUSDT", "ETHUSDT"]:
    df.loc[idx[:,symbol], "Ret_1"] = df.loc[idx[:,symbol], "Close"].pct_change().shift(-1)
df

Open      High       Low     Close   Volume  \
Open Time           Symbol                                                     
2024-11-05 13:24:00 BTCUSDT  68710.01  68789.68  68710.01  68736.92  0.29141   
                    ETHUSDT   2438.40   2440.11   2438.40   2440.11  3.47320   
2024-11-05 13:25:00 BTCUSDT  68736.00  68755.48  21250.00  68750.26  0.32806   
                    ETHUSDT   2438.81   2439.92   2438.81   2439.53  3.22890   
2024-11-05 13:26:00 BTCUSDT  68746.87  69039.27  68666.48  68748.01  0.48238   
...                               ...       ...       ...       ...      ...   
2024-12-21 22:58:00 ETHUSDT   3323.34   3326.53   3323.34   3326.53  1.38570   
2024-12-21 22:59:00 BTCUSDT  97094.05  97108.00  97094.05  97108.00  0.03038   
                    ETHUSDT   3326.71   3329.00   3326.71   3329.00  4.30580   
2024-12-21 23:00:00 BTCUSDT  97108.04  97131.59  97108.04  97131.59  0.07146   
                    ETHUSDT   3328.33   3330.48   3328.33   3330.48  4.73910   

                             Quote Asset Volume  Number of Trades  \
Open Time           Symbol                                          
2024-11-05 13:24:00 BTCUSDT        20032.192563              56.0   
                    ETHUSDT         8473.003310              27.0   
2024-11-05 13:25:00 BTCUSDT        22434.253383              73.0   
                    ETHUSDT         7876.714896              36.0   
2024-11-05 13:26:00 BTCUSDT        33174.664570             131.0   
...                                         ...               ...   
2024-12-21 22:58:00 ETHUSDT         4606.946458              21.0   
2024-12-21 22:59:00 BTCUSDT         2949.964025              16.0   
                    ETHUSDT        14329.616799              55.0   
2024-12-21 23:00:00 BTCUSDT         6940.328925              36.0   
                    ETHUSDT        15780.074164              58.0   

                             Taker Buy Base Asset Volume  \
Open Time           Symbol                                 
2024-11-05 13:24:00 BTCUSDT                      0.29141   
                    ETHUSDT                      3.17170   
2024-11-05 13:25:00 BTCUSDT                      0.10787   
                    ETHUSDT                      3.04850   
2024-11-05 13:26:00 BTCUSDT                      0.30178   
...                                                  ...   
2024-12-21 22:58:00 ETHUSDT                      1.38570   
2024-12-21 22:59:00 BTCUSDT                      0.03038   
                    ETHUSDT                      4.30580   
2024-12-21 23:00:00 BTCUSDT                      0.07046   
                    ETHUSDT                      4.73220   

                             Taker Buy Quote Asset Volume     Ret_1  
Open Time           Symbol                                           
2024-11-05 13:24:00 BTCUSDT                  20032.192563  0.000194  
                    ETHUSDT                   7737.543536       NaN  
2024-11-05 13:25:00 BTCUSDT                   7415.894900 -0.000033  
                    ETHUSDT                   7436.569564       NaN  
2024-11-05 13:26:00 BTCUSDT                  20755.789572  0.000031  
...                                                   ...       ...  
2024-12-21 22:58:00 ETHUSDT                   4606.946458       NaN  
2024-12-21 22:59:00 BTCUSDT                   2949.964025  0.000243  
                    ETHUSDT                  14329.616799       NaN  
2024-12-21 23:00:00 BTCUSDT                   6843.220885       NaN  
                    ETHUSDT                  15757.108687       NaN  

[133634 rows x 10 columns]